In [1]:
import torch
import numpy as np
import pandas as pd
from vit_pytorch import ViT
from dataloader import testing_dataset
from util import util
from torch.nn.functional import sigmoid
from torch.utils.data import Dataset, DataLoader

In [2]:
dataset = DataLoader(testing_dataset, shuffle=True)

In [3]:
torch.cuda.is_available()

True

In [4]:
vision_model = ViT(
    image_size=128 * 3,
    patch_size=32,
    num_classes=39,
    dim=512,
    depth=6,
    heads=16,
    mlp_dim=1024,
    # dropout = 0.1,
    # emb_dropout = 0.1
).to('cuda')

In [5]:
model_dict = torch.load('./saves/exp_7/model_epoch_100.pth')
model_dict.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss', 'testing_loss'])

In [6]:
vision_model.load_state_dict(model_dict['model_state_dict'])
vision_model.eval();

In [7]:
sample = next(iter(dataset))

In [8]:
with torch.no_grad():
    probs = vision_model(sample[0])

In [9]:
probs

tensor([[ 1.3897,  1.2626, -1.4595, -1.1851, -1.3380, -1.5432, -2.1853, -2.1191,
         -1.4460, -0.2281, -1.5515, -0.8435, -2.0845, -1.7886, -1.5234, -2.4760,
         -2.0175, -1.7185, -1.7176, -1.7110, -2.3173, -1.7355, -2.2722, -2.2099,
         -2.5214, -2.3524, -2.5703, -2.2753, -2.5015, -2.5356, -2.7017, -2.4688,
         -2.2861, -2.9755, -2.5537, -2.1075, -2.5259, -2.4284, -2.3472]],
       device='cuda:0')

In [10]:
sigmoid(probs)

tensor([[0.8005, 0.7795, 0.1885, 0.2341, 0.2078, 0.1761, 0.1011, 0.1073, 0.1906,
         0.4432, 0.1749, 0.3008, 0.1106, 0.1432, 0.1790, 0.0776, 0.1174, 0.1521,
         0.1522, 0.1530, 0.0897, 0.1499, 0.0934, 0.0989, 0.0744, 0.0869, 0.0711,
         0.0932, 0.0758, 0.0734, 0.0629, 0.0781, 0.0923, 0.0485, 0.0722, 0.1084,
         0.0741, 0.0810, 0.0873]], device='cuda:0')

In [85]:
@torch.no_grad()
def calculate_precession(y_hat: torch.Tensor, y: torch.Tensor):
    y = y.cpu()
    y_hat = y_hat.cpu()
    
    # y_hat = y_hat if y_hat.dim() == 2 else y_hat.unsqueeze(dim=0)
    # y = y if y.dim() == 2 else y.unsqueeze(dim=0)

    classes_hat = y_hat.clone().apply_(lambda x: 1 if x > .7 else 0)
    y_complement = y.clone().apply_(lambda x: 0 if x == 1 else 0)
    return y.sum(dim=-1)

In [86]:
sample[1]

tensor([[1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]], device='cuda:0')

In [87]:
calculate_precession(probs, sample[1])

tensor([2.])

In [66]:
_.shape

torch.Size([1, 39])

In [46]:
sample[1].sum(dim=0)

tensor([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.], device='cuda:0')